# データサイエンス100本ノック（構造化データ加工編） - SQL

## はじめに
- データベースはPostgreSQL13です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名: dsdojo_db
  - ユーザ名：padawan
  - パスワード:padawan12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します（設問にも出力件数を記載）
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - その場合、作業結果は消えますがファイルをGitHubから取り直してください
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

There's a new jupysql version available (0.10.9), you're running 0.7.2. To upgrade: pip install jupysql --upgrade


# 使い方
- セルの先頭に%%sqlと記載し、２行目以降にSQLを記述することでJupyterからPostgreSQLに対しSQLを実行できます。

In [6]:
%%sql
SELECT 'このように実行できます' AS sample;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


sample
このように実行できます


# データ加工100本ノック

---
> S-001: レシート明細データ（receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [4]:
%%sql
SELECT * FROM receipt LIMIT 10;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


解答をみると改行したほうが良さそう。

In [7]:
%%sql

SELECT 
    * 
FROM receipt 
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> S-002: レシート明細データ（receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [6]:
%%sql

SELECT
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,customer_id,product_cd,amount
20181103,CS006214000001,P070305012,158
20181118,CS008415000097,P070701017,81
20170712,CS028414000014,P060101005,170
20190205,ZZ000000000000,P050301001,25
20180821,CS025415000050,P060102007,90
20190605,CS003515000195,P050102002,138
20181205,CS024514000042,P080101005,30
20190922,CS040415000178,P070501004,128
20170504,ZZ000000000000,P071302010,770
20191010,CS027514000015,P071101003,680


---
> S-003: レシート明細データ（receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [7]:
%%sql

SELECT
    sales_ymd AS sales_date,
    customer_id,
    product_cd,
    amount
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_date,customer_id,product_cd,amount
20181103,CS006214000001,P070305012,158
20181118,CS008415000097,P070701017,81
20170712,CS028414000014,P060101005,170
20190205,ZZ000000000000,P050301001,25
20180821,CS025415000050,P060102007,90
20190605,CS003515000195,P050102002,138
20181205,CS024514000042,P080101005,30
20190922,CS040415000178,P070501004,128
20170504,ZZ000000000000,P071302010,770
20191010,CS027514000015,P071101003,680


---
> S-004: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [11]:
%%sql

SELECT 
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM receipt
WHERE customer_id = 'CS018205000001'
;

*  postgresql://padawan:***@db:5432/dsdojo_db
12 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


みやすさ的に条件も一行下げたほうが良さそう。

---
> S-005: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [14]:
%%sql

SELECT
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM receipt
WHERE
    customer_id = 'CS018205000001'
    AND
    amount >= 1000
;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> S-006: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [15]:
%%sql

SELECT
    sales_ymd,
    customer_id,
    product_cd,
    quantity,
    amount
FROM receipt
WHERE
    customer_id = 'CS018205000001'
    AND
    (amount >= 1000 OR quantity >= 5)
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


sales_ymd,customer_id,product_cd,quantity,amount
20180911,CS018205000001,P071401012,1,2200
20180414,CS018205000001,P060104007,6,600
20170614,CS018205000001,P050206001,5,990
20190226,CS018205000001,P071401020,1,2200
20180911,CS018205000001,P071401005,1,1100


---
> S-007: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [20]:
%%sql

SELECT
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM receipt
WHERE
    customer_id = 'CS018205000001'
    -- コメントアウトしたものも結果は同じ
    -- AND 1000 <= amount 
    -- AND amount <= 2000
    AND amount BETWEEN 1000 AND 2000
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401005,1100


---
> S-008: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [21]:
%%sql

SELECT
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM receipt
WHERE
    customer_id = 'CS018205000001'
    AND product_cd != 'P071401019'
;

*  postgresql://padawan:***@db:5432/dsdojo_db
12 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102
20180414,CS018205000001,P071101002,278
20190226,CS018205000001,P070902035,168
20190924,CS018205000001,P060805001,495
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> S-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
>
> `SELECT * FROM store WHERE NOT (prefecture_cd = '13' OR floor_area > 900)`

In [22]:
%%sql

SELECT 
    * 
FROM store 
WHERE 
    prefecture_cd != '13'
    AND floor_area <= 900
;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.66122,808.0


---
> S-010: 店舗データ（store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [24]:
%%sql

SELECT
    *
FROM store
WHERE
    -- 多分下のでもあってそう
    -- LEFT(store_cd, 3) = 'S14'
    store_cd LIKE 'S14%'
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> S-011: 顧客データ（customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [3]:
%%sql

SELECT
    *
FROM customer
WHERE
    customer_id LIKE '%1'
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> S-012: 店舗データ（store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [6]:
%%sql

SELECT
    *
FROM store
WHERE
    address LIKE '%横浜市%'
;

*  postgresql://padawan:***@db:5432/dsdojo_db
11 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> S-013: 顧客データ（customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [13]:
%%sql

SELECT 
    * 
FROM customer 
WHERE 
    status_cd ~ '^[A-F]' 
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> S-014: 顧客データ（customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [14]:
%%sql

SELECT 
    * 
FROM customer 
WHERE 
    status_cd ~ '[1-9]$' 
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> S-015: 顧客データ（customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [15]:
%%sql

SELECT 
    * 
FROM customer 
WHERE 
    status_cd ~ '^[A~F].*[1-9]$' 
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5
CS001514000191,高倉 礼子,1,女性,1965-11-04,53,144-0046,東京都大田区東六郷**********,S13001,20160628,A-20100731-1
CS008315000012,須藤 沙知絵,9,不明,1987-08-24,31,157-0074,東京都世田谷区大蔵**********,S13008,20151226,A-20090320-8
CS004413000091,小田 めぐみ,9,不明,1973-02-15,46,176-0024,東京都練馬区中村**********,S13004,20150921,A-20080512-1
CS039413000264,尾崎 菜摘,1,女性,1970-05-18,48,167-0051,東京都杉並区荻窪**********,S13039,20150621,A-20100221-4
CS014413000002,荻原 さゆり,1,女性,1970-03-20,49,264-0032,千葉県千葉市若葉区みつわ台**********,S12014,20150517,A-20080414-4


---
> S-016: 店舗データ（store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [19]:
%%sql

SELECT
    *
FROM store
WHERE
    tel_no ~ '^[0-9]{3}-[0-9]{3}-[0-9]{4}$'
;

*  postgresql://padawan:***@db:5432/dsdojo_db
34 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> S-017: 顧客データ（customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [20]:
%%sql

SELECT
    *
FROM customer
ORDER BY
    birth_day
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0
CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0


---
> S-018: 顧客データ（customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [21]:
%%sql

SELECT
    *
FROM customer
ORDER BY
    birth_day DESC
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> S-019: レシート明細データ（receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [26]:
%%sql

SELECT
    customer_id,
    amount,
    RANK() OVER
    (
        ORDER BY amount DESC
    ) AS rank
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,amount,rank
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
ZZ000000000000,5480,4
CS015515000034,5480,4
ZZ000000000000,5480,4
CS021515000089,5440,7
ZZ000000000000,5440,7
CS030515000189,5280,9
ZZ000000000000,5280,9


---
> S-020: レシート明細データ（receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [27]:
%%sql

SELECT
    customer_id,
    amount,
    ROW_NUMBER() OVER
    (
        ORDER BY amount DESC
    ) AS rank
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,amount,rank
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
ZZ000000000000,5480,4
CS015515000034,5480,5
ZZ000000000000,5480,6
ZZ000000000000,5440,7
CS021515000089,5440,8
CS001412000160,5280,9
CS020414000081,5280,10


---
> S-021: レシート明細データ（receipt）に対し、件数をカウントせよ。

In [28]:
%%sql

SELECT
    COUNT(*)
FROM receipt
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
104681


---
> S-022: レシート明細データ（receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [32]:
%%sql

SELECT
    COUNT(DISTINCT customer_id)
FROM receipt
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
8307


---
> S-023: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [36]:
%%sql

SELECT
    store_cd,
    SUM(amount) AS sum_amount,
    SUM(quantity) AS sum_quantity
FROM receipt
GROUP BY store_cd
;

*  postgresql://padawan:***@db:5432/dsdojo_db
52 rows affected.


store_cd,sum_amount,sum_quantity
S12007,638761,2099
S13017,748221,2376
S13043,587895,1881
S13052,100314,250
S13016,793773,2432
S14027,714550,2303
S13009,808870,2486
S14022,651328,2047
S13019,827833,2541
S13039,611888,1981


---
> S-024: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [37]:
%%sql

SELECT
    customer_id,
    MAX(sales_ymd)
FROM receipt
GROUP BY customer_id
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,max
CS001311000059,20180211
CS004614000122,20181228
CS003512000043,20180106
CS011615000061,20190503
CS029212000033,20180621
CS007515000119,20190511
CS034515000123,20190708
CS004315000058,20170517
CS026414000014,20190720
CS001615000099,20170729


---
> S-025: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [38]:
%%sql

SELECT
    customer_id,
    MIN(sales_ymd)
FROM receipt
GROUP BY customer_id
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,min
CS001311000059,20180211
CS004614000122,20181228
CS003512000043,20180106
CS011615000061,20190503
CS029212000033,20170318
CS007515000119,20170201
CS034515000123,20170527
CS004315000058,20170517
CS026414000014,20170718
CS001615000099,20170729


---
> S-026: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [6]:
%%sql

SELECT
    customer_id,
    MAX(sales_ymd),
    MIN(sales_ymd)
FROM receipt
GROUP BY customer_id
HAVING MAX(sales_ymd) != MIN(sales_ymd)
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,max,min
CS029212000033,20180621,20170318
CS007515000119,20190511,20170201
CS034515000123,20190708,20170527
CS026414000014,20190720,20170718
CS010515000082,20181204,20180518
CS019315000045,20170920,20170423
CS008513000099,20190308,20170722
CS007615000070,20191025,20170929
CS025415000155,20191026,20170314
CS016414000063,20190617,20170109


---
> S-027: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [9]:
%%sql

SELECT
    store_cd,
    AVG(amount) AS avg_amount
FROM receipt
GROUP BY store_cd
ORDER BY avg_amount DESC
LIMIT 5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,avg_amount
S13052,402.8674698795180723
S13015,351.1119604316546763
S13003,350.9155188246097337
S14010,348.7912621359223301
S13001,348.4703862660944206


---
> S-028: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [12]:
%%sql

SELECT
    store_cd,
    -- 中央値を計算する関数はない
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY amount) AS amount_50per
FROM receipt
GROUP BY store_cd
ORDER BY amount_50per DESC
LIMIT 5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,amount_50per
S13052,190.0
S14010,188.0
S14050,185.0
S13003,180.0
S13018,180.0


---
> S-029: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [2]:
%%sql

WITH 
-- 店舗コードと商品コードの組み合わせでグループ化し個数をカウントして、
-- 一時的なテーブルproduct_cntに結果を保存
product_cnt AS (
    SELECT
        store_cd,
        product_cd,
        COUNT(1) AS mode_cnt
    FROM receipt
    GROUP BY
        store_cd,
        product_cd
),
-- product_cntを店舗コードごとにカウントで降順にランク付けしたテーブルをproduct_modeとしている。
product_mode AS (
    SELECT
        store_cd,
        product_cd,
        mode_cnt,
        RANK() OVER(PARTITION BY store_cd ORDER BY mode_cnt DESC) AS rnk
    FROM product_cnt
)
-- product_modeからrankが１、つまりもっともカウントが多いものを取り出している。
SELECT
    store_cd,
    product_cd,
    mode_cnt
FROM product_mode
WHERE
    rnk = 1
ORDER BY
    store_cd,
    product_cd
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


store_cd,product_cd,mode_cnt
S12007,P060303001,72
S12013,P060303001,107
S12014,P060303001,65
S12029,P060303001,92
S12030,P060303001,115
S13001,P060303001,67
S13002,P060303001,78
S13003,P071401001,65
S13004,P060303001,88
S13005,P040503001,36


---
> S-030: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [5]:
%%sql

SELECT
    store_cd,
    VAR_POP(amount) as var_amount
FROM receipt
GROUP BY store_cd
-- 降順のときにDESCを忘れない
ORDER BY var_amount DESC
LIMIT 5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,var_amount
S13052,440088.701311269173
S14011,306314.558163888889
S14034,296920.081011283873
S13001,295431.993329035348
S13015,295294.361115940880


---
> S-031: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [6]:
%%sql

SELECT
    store_cd,
    STDDEV_POP(amount) as std_amount
FROM receipt
GROUP BY store_cd
-- 降順のときにDESCを忘れない
ORDER BY std_amount DESC
LIMIT 5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,std_amount
S13052,663.391815830787
S14011,553.456916267101
S14034,544.903735545357
S13001,543.536561170484
S13015,543.409938366921


---
> S-032: レシート明細データ（receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [10]:
%%sql

SELECT
    PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY amount) AS amount_25per,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY amount) AS amount_25per,
    PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY amount) AS amount_25per,
    PERCENTILE_CONT(1.0) WITHIN GROUP(ORDER BY amount) AS amount_25per
FROM receipt
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


amount_25per,amount_25per_1,amount_25per_2,amount_25per_3
102.0,170.0,288.0,10925.0


---
> S-033: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [14]:
%%sql

SELECT
    store_cd,
    AVG(amount) AS avg_amount
FROM receipt
GROUP BY store_cd
-- HAVING句ではASでつけた別名は使えない。
HAVING AVG(amount) >= 330
;

*  postgresql://padawan:***@db:5432/dsdojo_db
13 rows affected.


store_cd,avg_amount
S13052,402.8674698795180723
S13019,330.2086158755484643
S13003,350.9155188246097337
S14045,330.0820734341252700
S13004,330.9439490445859873
S13001,348.4703862660944206
S14026,332.3405884723901653
S14010,348.7912621359223301
S13015,351.1119604316546763
S12013,330.1941299790356394


---
> S-034: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [17]:
%%sql

-- 複雑なクエリは一時的なテーブルを使う
WITH
customer_amount_sum AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    WHERE customer_id NOT LIKE 'Z%'
    GROUP BY customer_id
)
SELECT
    AVG(sum_amount)
FROM customer_amount_sum
;
    

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


avg
2547.7422345292559595


---
> S-035: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [23]:
%%sql


WITH
customer_amount_sum AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    WHERE customer_id NOT LIKE 'Z%'
    GROUP BY customer_id
)
-- 入れ子問い合わせを用いると便利
SELECT
    customer_id,
    sum_amount
FROM customer_amount_sum
WHERE
    sum_amount >= (SELECT
                       AVG(sum_amount)
                   FROM customer_amount_sum)
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount
CS029212000033,3604
CS007515000119,7157
CS034515000123,3699
CS026414000014,6671
CS007615000070,2975
CS016414000063,6207
CS012514000018,2562
CS029515000142,3420
CS015215000021,3090
CS039814000011,8031


---
> S-036: レシート明細データ（receipt）と店舗データ（store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [26]:
%%sql

SELECT
    *
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


In [28]:
%%sql

SELECT
    *
FROM store
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
S13051,板橋大原店,13,東京都,東京都板橋区大原町,トウキョウトイタバシクオオハラチョウ,03-0123-4029,139.698,35.76788,1045.0
S13015,南砂店,13,東京都,東京都江東区南砂二丁目,トウキョウトコウトウクミナミスナニチョウメ,03-0123-4014,139.8229,35.67066,1337.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S13044,南六郷店,13,東京都,東京都大田区南六郷二丁目,トウキョウトオオタクミナミロクゴウニチョウメ,03-0123-4028,139.7207,35.54604,1379.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0


In [29]:
%%sql

-- 上の結果からstore_cdで結合できる。
SELECT
    receipt.*,
    store.store_name
FROM receipt
INNER JOIN store
ON receipt.store_cd = store.store_cd
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,成城店
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,二ツ橋店
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,新山下店
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,大和店
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,狛江店
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30,三田店
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128,長津田店
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770,十条仲原店
20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680,南藤沢店


---
> S-037: 商品データ（product）とカテゴリデータ（category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [33]:
%%sql

SELECT
    p.*,
    c.category_small_cd
FROM product p
INNER JOIN category c
ON 
    p.category_small_cd = c.category_small_cd
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_cd_1
P040101001,04,0401,040101,198,149,040101
P040101002,04,0401,040101,218,164,040101
P040101003,04,0401,040101,230,173,040101
P040101004,04,0401,040101,248,186,040101
P040101005,04,0401,040101,268,201,040101
P040101006,04,0401,040101,298,224,040101
P040101007,04,0401,040101,338,254,040101
P040101008,04,0401,040101,420,315,040101
P040101009,04,0401,040101,498,374,040101
P040101010,04,0401,040101,580,435,040101


---
> S-038: 顧客データ（customer）とレシート明細データ（receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [40]:
%%sql

WITH
customer_sum_amount AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    GROUP BY customer_id
),
female_member_customer AS(
    SELECT
        customer_id
    FROM customer
    WHERE
        gender_cd = '1'
        AND customer_id NOT LIKE 'Z%'
)
SELECT
    fmc.customer_id,
    -- COALESCEはNoneを何で埋めるか指定できる。
    COALESCE(csa.sum_amount, 0)
FROM female_member_customer fmc
LEFT OUTER JOIN customer_sum_amount csa
ON fmc.customer_id = csa.customer_id
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,coalesce
CS021313000114,0
CS031415000172,5088
CS028811000001,0
CS001215000145,875
CS015414000103,3122
CS033513000180,868
CS035614000014,0
CS011215000048,3444
CS009413000079,0
CS040412000191,210


---
> S-039: レシート明細データ（receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [45]:
%%sql

WITH 
member_customer AS (
    SELECT
        customer_id,
        sales_ymd,
        amount
    FROM receipt
    WHERE
        customer_id NOT LIKE 'Z%'
),
top_20_sale_days AS (
    SELECT
        customer_id,
        COUNT(DISTINCT sales_ymd) as count_days
    FROM member_customer
    GROUP BY customer_id
    ORDER BY count_days DESC
    LIMIT 20
),
top_20_amount AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM member_customer
    GROUP BY customer_id
    ORDER BY sum_amount DESC
    LIMIT 20
)
SELECT
    COALESCE(t2sd.customer_id, t2a.customer_id) customer_id,
    t2sd.count_days,
    t2a.sum_amount
FROM top_20_sale_days t2sd
FULL OUTER JOIN top_20_amount t2a
ON t2sd.customer_id = t2a.customer_id
;

*  postgresql://padawan:***@db:5432/dsdojo_db
34 rows affected.


customer_id,count_days,sum_amount
CS040214000008,23,None
CS015415000185,22,20153
CS010214000010,22,18585
CS028415000007,21,19127
CS010214000002,21,None
CS017415000097,20,23086
CS016415000141,20,18372
CS021514000045,19,None
CS022515000226,19,None
CS031414000051,19,19202


---
> S-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（store）と商品データ（product）を直積し、件数を計算せよ。

In [48]:
%%sql

SELECT
    COUNT(1)
FROM store
CROSS JOIN product
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
531590


---
> S-041: レシート明細データ（receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [6]:
%%sql

WITH
sales_amount AS (
    SELECT
        sales_ymd,
        SUM(amount) as sum_amount
    FROM receipt
    GROUP BY sales_ymd
)
SELECT
    sales_ymd,
    sum_amount,
    LAG(sales_ymd, 1) OVER(ORDER BY sales_ymd) AS lag_ymd,
    LAG(sum_amount, 1) OVER(ORDER BY sales_ymd) AS prev_sum_amount,
    sum_amount - LAG(sum_amount, 1) OVER(ORDER BY sales_ymd) AS diff_sum_amount
FROM sales_amount
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sum_amount,lag_ymd,prev_sum_amount,diff_sum_amount
20170101,33723,None,None,None
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338
20170104,36165,20170103,27503,8662
20170105,37830,20170104,36165,1665
20170106,32387,20170105,37830,-5443
20170107,23415,20170106,32387,-8972
20170108,24737,20170107,23415,1322
20170109,26718,20170108,24737,1981
20170110,20143,20170109,26718,-6575


---
> S-042: レシート明細データ（receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [8]:
%%sql

WITH
sales_amount AS (
    SELECT
        sales_ymd,
        SUM(amount) as sum_amount
    FROM receipt
    GROUP BY sales_ymd
)
SELECT
    sales_ymd,
    sum_amount,
    LAG(sales_ymd, 1) OVER (ORDER BY sales_ymd) AS lag_ymd_1,
    LAG(sum_amount, 1) OVER(ORDER BY sales_ymd) AS pre_1_v_sum_amount_,
    LAG(sales_ymd, 2) OVER (ORDER BY sales_ymd) AS lag_ymd_2,
    LAG(sum_amount, 2) OVER(ORDER BY sales_ymd) AS prev_2_sum_amount,
    LAG(sales_ymd, 3) OVER (ORDER BY sales_ymd) AS lag_ymd_3,
    LAG(sum_amount, 3) OVER(ORDER BY sales_ymd) AS prev_3_sum_amount
FROM sales_amount
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sum_amount,lag_ymd_1,pre_1_v_sum_amount_,lag_ymd_2,prev_2_sum_amount,lag_ymd_3,prev_3_sum_amount
20170101,33723,None,None,None,None,None,None
20170102,24165,20170101,33723,None,None,None,None
20170103,27503,20170102,24165,20170101,33723,None,None
20170104,36165,20170103,27503,20170102,24165,20170101,33723
20170105,37830,20170104,36165,20170103,27503,20170102,24165
20170106,32387,20170105,37830,20170104,36165,20170103,27503
20170107,23415,20170106,32387,20170105,37830,20170104,36165
20170108,24737,20170107,23415,20170106,32387,20170105,37830
20170109,26718,20170108,24737,20170107,23415,20170106,32387
20170110,20143,20170109,26718,20170108,24737,20170107,23415


---
> S-043: レシート明細データ（receipt）と顧客データ（customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [15]:
%%sql

-- 「作成せよ」なのでCREATE句を付ける。
CREATE TABLE sales_summary AS
    WITH gender_group_amount AS (
        SELECT
            TRUNC(c.age / 10) * 10 AS era,
            c.gender_cd,
            SUM(r.amount) AS sum_amount
        FROM receipt r
        JOIN customer c
        ON r.customer_id = c.customer_id
        GROUP BY c.gender_cd, c.age 
    )
    SELECT
        era,
        SUM(CASE WHEN gender_cd = '0' THEN sum_amount END) AS male,
        SUM(CASE WHEN gender_cd = '1' THEN sum_amount END) AS female,
        SUM(CASE WHEN gender_cd = '9' THEN sum_amount END) AS unknown
    FROM gender_group_amount
    GROUP BY era
    ORDER BY era
;

SELECT
    *
FROM sales_summary
;

*  postgresql://padawan:***@db:5432/dsdojo_db
9 rows affected.
9 rows affected.


era,male,female,unknown
10.0,1591,149836,4317
20.0,72940,1363724,44328
30.0,177322,693047,50441
40.0,19355,9320791,483512
50.0,54320,6685192,342923
60.0,272469,987741,71418
70.0,13435,29764,2427
80.0,46360,262923,5111
90.0,None,6260,None


---
> S-044: 043で作成した売上サマリデータ（sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [18]:
%%sql

-- UNION ALLを使って横持ちから縦持ちに変換する。
SELECT
    era,
    '00' AS gender_cd,
    male AS sum_amount
FROM sales_summary
UNION ALL
SELECT
    era,
    '01' AS gender_cd,
    female AS sum_amount
FROM sales_summary
UNION ALL
SELECT
    era,
    '99' AS gender_cd,
    unknown AS sum_amount
FROM sales_summary
;   

*  postgresql://padawan:***@db:5432/dsdojo_db
27 rows affected.


era,gender_cd,sum_amount
10.0,00,1591
20.0,00,72940
30.0,00,177322
40.0,00,19355
50.0,00,54320
60.0,00,272469
70.0,00,13435
80.0,00,46360
90.0,00,None
10.0,01,149836


---
> S-045: 顧客データ（customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [20]:
%%sql

-- TO_CHAR関数で文字列変換
SELECT
    customer_id,
    TO_CHAR(birth_day, 'YYYYMMDD') AS birth_day
FROM customer
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,birth_day
CS021313000114,19810429
CS037613000071,19520401
CS031415000172,19761004
CS028811000001,19330327
CS001215000145,19950329
CS020401000016,19740915
CS015414000103,19770809
CS029403000008,19730817
CS015804000004,19310502
CS033513000180,19620711


---
> S-046: 顧客データ（customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [21]:
%%sql

SELECT
    customer_id,
    TO_DATE(application_date, 'YYYYMMDD') AS application_date
FROM customer
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,application_date
CS021313000114,2015-09-05
CS037613000071,2015-04-14
CS031415000172,2015-05-29
CS028811000001,2016-01-15
CS001215000145,2017-06-05
CS020401000016,2015-02-25
CS015414000103,2015-07-22
CS029403000008,2015-05-15
CS015804000004,2015-06-07
CS033513000180,2015-07-28


---
> S-047: レシート明細データ（receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [23]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    TO_DATE(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD') AS sales_ymd
FROM receipt
LIMIT 10
; 

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,sales_ymd
112,1,2018-11-03
1132,2,2018-11-18
1102,1,2017-07-12
1132,1,2019-02-05
1102,2,2018-08-21
1112,1,2019-06-05
1102,2,2018-12-05
1102,1,2019-09-22
1112,2,2017-05-04
1102,1,2019-10-10


---
> S-048: レシート明細データ（receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [24]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    CAST(TO_TIMESTAMP(sales_epoch) AS DATE) AS sales_epoch
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,sales_epoch
112,1,2018-11-03
1132,2,2018-11-18
1102,1,2017-07-12
1132,1,2019-02-05
1102,2,2018-08-21
1112,1,2019-06-05
1102,2,2018-12-05
1102,1,2019-09-22
1112,2,2017-05-04
1102,1,2019-10-10


---
> S-049: レシート明細データ（receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [27]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    EXTRACT(YEAR FROM TO_TIMESTAMP(sales_epoch)) AS sales_year
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,sales_year
112,1,2018
1132,2,2018
1102,1,2017
1132,1,2019
1102,2,2018
1112,1,2019
1102,2,2018
1102,1,2019
1112,2,2017
1102,1,2019


---
> S-050: レシート明細データ（receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [29]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    TO_CHAR(EXTRACT(MONTH FROM TO_TIMESTAMP(sales_epoch)), 'FM00') AS sales_month
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,sales_month
112,1,11
1132,2,11
1102,1,07
1132,1,02
1102,2,08
1112,1,06
1102,2,12
1102,1,09
1112,2,05
1102,1,10


---
> S-051: レシート明細データ（receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [6]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    TO_CHAR(EXTRACT(DAY FROM TO_TIMESTAMP(sales_epoch)), 'FM00') AS sales_day
FROM receipt
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


receipt_no,receipt_sub_no,sales_day
112,1,03
1132,2,18
1102,1,12
1132,1,05
1102,2,21
1112,1,05
1102,2,05
1102,1,22
1112,2,04
1102,1,10


---
> S-052: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [11]:
%%sql

WITH member_sum_amount AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    WHERE customer_id NOT LIKE 'Z%'
    GROUP BY customer_id
)
SELECT
    customer_id,
    sum_amount,
    CASE WHEN sum_amount <= 2000 THEN 0
		 WHEN sum_amount > 2000 THEN 1
    END AS sum_amount_bool
FROM member_sum_amount
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,sum_amount_bool
CS001311000059,2302,1
CS004614000122,248,0
CS003512000043,298,0
CS011615000061,246,0
CS029212000033,3604,1
CS007515000119,7157,1
CS034515000123,3699,1
CS004315000058,490,0
CS026414000014,6671,1
CS001615000099,768,0


---
> S-053: 顧客データ（customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [21]:
%%sql

WITH  bool_tokyo_table AS (
    SELECT
        customer_id,
        postal_cd,
        CASE 
            -- WHEN postal_cd LIKE '^[100~209]' THEN 1 みたいなことをしたい
            -- SUBSTR関数を使って部分文字列を取得
            WHEN CAST(SUBSTR(postal_cd, 1, 3) AS INTEGER) BETWEEN 100 AND 209 THEN 1
            ELSE 0    
        END AS bool_tokyo
    FROM customer
)
SELECT
    b.bool_tokyo,
    COUNT(DISTINCT r.customer_id) as count_customer
FROM bool_tokyo_table b
JOIN receipt r
ON b.customer_id = r.customer_id
GROUP BY b.bool_tokyo
;

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


bool_tokyo,count_customer
0,3906
1,4400


---
> S-054: 顧客データ（customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [23]:
%%sql

SELECT
    customer_id,
    address,
    CASE
        WHEN address LIKE '埼玉県%' THEN 11
        WHEN address LIKE '千葉県%' THEN 12
        WHEN address LIKE '東京都%' THEN 13
        WHEN address LIKE '神奈川県%' THEN 14
    END AS address_code
FROM customer
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,address,address_code
CS021313000114,神奈川県伊勢原市粟窪**********,14
CS037613000071,東京都江東区南砂**********,13
CS031415000172,東京都渋谷区代々木**********,13
CS028811000001,神奈川県横浜市泉区和泉町**********,14
CS001215000145,東京都大田区仲六郷**********,13
CS020401000016,東京都板橋区若木**********,13
CS015414000103,東京都江東区北砂**********,13
CS029403000008,千葉県浦安市海楽**********,12
CS015804000004,東京都江東区北砂**********,13
CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> S-055: レシート明細（receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [35]:
%%sql

WITH sum_amount_table AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    GROUP BY customer_id
), -- 回答によると四分位範囲のテーブルを作成してクロス集計を使っている。
sales_pct AS (
    SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY sum_amount) AS pct25,
        PERCENTILE_CONT(0.50) WITHIN GROUP(ORDER BY sum_amount) AS pct50,
        PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY sum_amount) AS pct75
    FROM
        sum_amount_table
)
SELECT
    s.customer_id,
    s.sum_amount,
    CASE
        WHEN s.sum_amount < p.pct25 THEN 1
        WHEN p.pct25 <= s.sum_amount AND s.sum_amount < p.pct50 THEN 2
        WHEN p.pct50 <= s.sum_amount AND s.sum_amount < p.pct75 THEN 3
        WHEN p.pct75 <= s.sum_amount THEN 4
    END AS pct_group
FROM sum_amount_table s
CROSS JOIN sales_pct p
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,pct_group
CS001311000059,2302,3
CS004614000122,248,1
CS003512000043,298,1
CS011615000061,246,1
CS029212000033,3604,3
CS007515000119,7157,4
CS034515000123,3699,4
CS004315000058,490,1
CS026414000014,6671,4
CS001615000099,768,2


---
> S-056: 顧客データ（customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [4]:
%%sql

SELECT
    customer_id,
    birth_day,
    CASE
        WHEN age >= 60 THEN 60
        ELSE CAST(TRUNC(age / 10) * 10 AS INTEGER)
    END as era
FROM customer
-- GROUP BYがいる理由は不明
GROUP BY
    customer_id,
    birth_day
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,birth_day,era
CS001413000478,1953-01-29,60
CS001413000794,1939-10-09,60
CS001415000389,1933-10-08,60
CS001501000030,1940-07-01,60
CS001512000067,1959-01-04,60
CS001512000326,1959-02-17,60
CS001512000368,1959-03-23,60
CS001512000417,1959-01-20,60
CS001512000436,1953-02-13,60
CS001513000035,1958-12-10,60


---
> S-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [7]:
%%sql

SELECT
    customer_id,
    birth_day,
    -- ||は文字列の連結に使う。
    gender_cd ||
    CASE
        WHEN age >= 60 THEN '60'
        ELSE TO_CHAR(CAST(TRUNC(age / 10) * 10 AS INTEGER), 'FM00')
    END as gender_era
FROM customer
-- GROUP BYがいる理由は不明
GROUP BY
    customer_id,
    birth_day
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,birth_day,gender_era
CS001105000001,2000-01-14,010
CS001112000009,2006-08-24,110
CS001112000019,2001-01-31,110
CS001112000021,2001-12-15,110
CS001112000023,2004-01-26,110
CS001112000024,2001-01-16,110
CS001112000029,2005-01-24,110
CS001112000030,2003-03-02,110
CS001113000004,2003-02-22,110
CS001113000010,2005-05-09,110


---
> S-058: 顧客データ（customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [8]:
%%sql

-- ダミー変数化の手法の一つとしてOne-Hotベクトル表現がある
SELECT
    customer_id,
    CASE
        WHEN gender_cd = '0' THEN 1
        ELSE 0
    END AS dammy_male,
    CASE
        WHEN gender_cd = '1' THEN 1
        ELSE 0
    END AS dammy_female,
    CASE
        WHEN gender_cd = '9' THEN 1
        ELSE 0
    END AS dammy_unknow
FROM customer
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,dammy_male,dammy_female,dammy_unknow
CS021313000114,0,1,0
CS037613000071,0,0,1
CS031415000172,0,1,0
CS028811000001,0,1,0
CS001215000145,0,1,0
CS020401000016,1,0,0
CS015414000103,0,1,0
CS029403000008,1,0,0
CS015804000004,1,0,0
CS033513000180,0,1,0


---
> S-059: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [17]:
%%sql

WITH member_sum_amount AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    WHERE customer_id NOT LIKE 'Z%'
    GROUP BY customer_id
),
mean_std_amount AS (
    SELECT
        AVG(sum_amount) as avg_amount,
        STDDEV_POP(sum_amount) as std_amount
    FROM member_sum_amount
)
SELECT
    m.customer_id,
    m.sum_amount,
    (m.sum_amount - avg_amount) / std_amount AS standard_amount
FROM member_sum_amount m
CROSS JOIN mean_std_amount mean
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,standard_amount
CS001311000059,2302,-0.09032946448390523062
CS004614000122,248,-0.84533488878695858131
CS003512000043,298,-0.82695598361891930792
CS011615000061,246,-0.84607004499368015224
CS029212000033,3604,0.38825722609183744835
CS007515000119,7157,1.6942622273327082
CS034515000123,3699,0.42317714591111206778
CS004315000058,490,-0.75638098777364849812
CS026414000014,6671,1.5156192690993665
CS001615000099,768,-0.65419427503935013810


---
> S-060: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [20]:
%%sql

WITH member_sum_amount AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    WHERE customer_id NOT LIKE 'Z%'
    GROUP BY customer_id
),
max_min_amount AS (
    SELECT
        MAX(sum_amount) as max_amount,
        MIN(sum_amount) as min_amount
    FROM member_sum_amount
)
SELECT
    m.customer_id,
    m.sum_amount,
    -- 1.0をかけないとint型となってしまう
    1.0 * (m.sum_amount - min_amount) / (max_amount - min_amount) AS scaled_amount
FROM member_sum_amount m
CROSS JOIN max_min_amount
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,scaled_amount
CS001311000059,2302,0.09697601668404588113
CS004614000122,248,0.00773375043448036149
CS003512000043,298,0.00990615224191866528
CS011615000061,246,0.00764685436218282934
CS029212000033,3604,0.15354535974973931178
CS007515000119,7157,0.30791623218630517901
CS034515000123,3699,0.15767292318387208898
CS004315000058,490,0.01824817518248175182
CS026414000014,6671,0.28680048661800486618
CS001615000099,768,0.03032672923183872089


---
> S-061: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [5]:
%%sql

WITH member_sum_amount AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    WHERE customer_id NOT LIKE 'Z%'
    GROUP BY customer_id
)
SELECT
    customer_id,
    sum_amount,
    -- 0を常用対数化するため0.5を足す
    LOG10(sum_amount + 0.5) as log10_sum_amount
FROM member_sum_amount
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,log10_sum_amount
CS001311000059,2302,3.3621996388688865
CS004614000122,248,2.3953263930693509
CS003512000043,298,2.4749443354653879
CS011615000061,246,2.3918169236132488
CS029212000033,3604,3.5568450298595813
CS007515000119,7157,3.8547613566936362
CS034515000123,3699,3.5681430316577019
CS004315000058,490,2.6906390117159673
CS026414000014,6671,3.8242234903608168
CS001615000099,768,2.8856438718357639


---
> S-062: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [6]:
%%sql

WITH member_sum_amount AS (
    SELECT
        customer_id,
        SUM(amount) as sum_amount
    FROM receipt
    WHERE customer_id NOT LIKE 'Z%'
    GROUP BY customer_id
)
SELECT
    customer_id,
    sum_amount,
    LN(sum_amount + 0.5) as log10_sum_amount
FROM member_sum_amount
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,log10_sum_amount
CS001311000059,2302,7.7417507681294619
CS004614000122,248,5.5154428455366834
CS003512000043,298,5.6987699328326568
CS011615000061,246,5.5073619934827448
CS029212000033,3604,8.1899383438446333
CS007515000119,7157,8.8759160369722701
CS034515000123,3699,8.2159529543656056
CS004315000058,490,6.1954252790054178
CS026414000014,6671,8.8056000011824754
CS001615000099,768,6.6444405629786506


---
> S-063: 商品データ（product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [8]:
%%sql

SELECT
    product_cd,
    unit_price,
    unit_cost,
    unit_price - unit_cost AS unit_profit
FROM product
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,unit_price,unit_cost,unit_profit
P040101001,198,149,49
P040101002,218,164,54
P040101003,230,173,57
P040101004,248,186,62
P040101005,268,201,67
P040101006,298,224,74
P040101007,338,254,84
P040101008,420,315,105
P040101009,498,374,124
P040101010,580,435,145


---
> S-064: 商品データ（product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [13]:
%%sql

WITH product_profit AS (
    SELECT
        product_cd,
        COALESCE(unit_price, 0) AS unit_price,
        COALESCE(unit_cost, 0) AS unit_cost,
        unit_price - unit_cost AS unit_profit
    FROM product
),
product_progit_rate AS (
    SELECT
        1.0 * SUM(unit_profit) / SUM(unit_price) AS profit_rate
    FROM product_profit
    GROUP BY product_cd
)
SELECT
    AVG(profit_rate)
FROM product_progit_rate
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


avg
0.24911389885177001279


---
> S-065: 商品データ（product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [14]:
%%sql

WITH profit_rate_30 AS (
    SELECT
        product_cd,
        COALESCE(unit_price, 0) AS unit_price,
        COALESCE(unit_cost, 0) AS unit_cost,
        TRUNC(unit_cost / 0.7) AS unit_cost_profit30
    FROM product
)
SELECT
    *,
    (unit_cost_profit30 - unit_cost) / unit_cost_profit30 AS about_profit30
FROM profit_rate_30
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,unit_price,unit_cost,unit_cost_profit30,about_profit30
P040101001,198,149,212,0.29716981132075471698
P040101002,218,164,234,0.29914529914529914530
P040101003,230,173,247,0.29959514170040485830
P040101004,248,186,265,0.29811320754716981132
P040101005,268,201,287,0.29965156794425087108
P040101006,298,224,320,0.30000000000000000000
P040101007,338,254,362,0.29834254143646408840
P040101008,420,315,450,0.30000000000000000000
P040101009,498,374,534,0.29962546816479400749
P040101010,580,435,621,0.29951690821256038647


---
> S-066: 商品データ（product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [2]:
%%sql

WITH profit_rate_30 AS (
    SELECT
        product_cd,
        COALESCE(unit_price, 0) AS unit_price,
        COALESCE(unit_cost, 0) AS unit_cost,
        ROUND(unit_cost / 0.7) AS unit_cost_profit30
    FROM product
)
SELECT
    *,
    (unit_cost_profit30 - unit_cost) / unit_cost_profit30 AS about_profit30
FROM profit_rate_30
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,unit_price,unit_cost,unit_cost_profit30,about_profit30
P040101001,198,149,213,0.30046948356807511737
P040101002,218,164,234,0.29914529914529914530
P040101003,230,173,247,0.29959514170040485830
P040101004,248,186,266,0.30075187969924812030
P040101005,268,201,287,0.29965156794425087108
P040101006,298,224,320,0.30000000000000000000
P040101007,338,254,363,0.30027548209366391185
P040101008,420,315,450,0.30000000000000000000
P040101009,498,374,534,0.29962546816479400749
P040101010,580,435,621,0.29951690821256038647


---
> S-067: 商品データ（product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [3]:
%%sql

WITH profit_rate_30 AS (
    SELECT
        product_cd,
        COALESCE(unit_price, 0) AS unit_price,
        COALESCE(unit_cost, 0) AS unit_cost,
        CEILING(unit_cost / 0.7) AS unit_cost_profit30
    FROM product
)
SELECT
    *,
    (unit_cost_profit30 - unit_cost) / unit_cost_profit30 AS about_profit30
FROM profit_rate_30
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,unit_price,unit_cost,unit_cost_profit30,about_profit30
P040101001,198,149,213,0.30046948356807511737
P040101002,218,164,235,0.30212765957446808511
P040101003,230,173,248,0.30241935483870967742
P040101004,248,186,266,0.30075187969924812030
P040101005,268,201,288,0.30208333333333333333
P040101006,298,224,320,0.30000000000000000000
P040101007,338,254,363,0.30027548209366391185
P040101008,420,315,450,0.30000000000000000000
P040101009,498,374,535,0.30093457943925233645
P040101010,580,435,622,0.30064308681672025723


---
> S-068: 商品データ（product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [4]:
%%sql

SELECT
    product_cd,
    unit_price,
    TRUNC(unit_price * 1.1) AS tax_price
FROM product
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


product_cd,unit_price,tax_price
P040101001,198,217
P040101002,218,239
P040101003,230,253
P040101004,248,272
P040101005,268,294
P040101006,298,327
P040101007,338,371
P040101008,420,462
P040101009,498,547
P040101010,580,638


---
> S-069: レシート明細データ（receipt）と商品データ（product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [15]:
%%sql

WITH customer_sum_amount AS (
    SELECT
        r.customer_id,
        SUM(r.amount) as sum_amount
    FROM receipt r
    GROUP BY r.customer_id
),
category07_sum_amount AS (
    SELECT
        r.customer_id,
        SUM(r.amount) as sum07_amount
    FROM receipt r
    JOIN product p
    ON r.product_cd = p.product_cd
    WHERE p.category_major_cd = '07'
    GROUP BY r.customer_id
)
SELECT
    c.customer_id,
    c.sum_amount,
    c07.sum07_amount,
    (1.0 * c07.sum07_amount / c.sum_amount) AS rate
FROM customer_sum_amount c
JOIN category07_sum_amount c07
ON c.customer_id = c07.customer_id
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,sum07_amount,rate
CS001311000059,2302,102,0.04430929626411815812
CS011615000061,246,98,0.39837398373983739837
CS029212000033,3604,3604,1.00000000000000000000
CS007515000119,7157,2832,0.39569652088864049183
CS034515000123,3699,1202,0.32495268991619356583
CS026414000014,6671,3142,0.47099385399490331285
CS001615000099,768,318,0.41406250000000000000
CS010515000082,1482,553,0.37314439946018893387
CS019315000045,813,380,0.46740467404674046740
CS008513000099,1322,210,0.15885022692889561271


---
> S-070: レシート明細データ（receipt）の売上日（sales_ymd）に対し、顧客データ（customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [43]:
%%sql

-- あってそうだけど解答と出力の順番が違う
WITH date_table AS (
    SELECT
        r.customer_id AS customer_id,
        TO_TIMESTAMP(CAST(r.sales_ymd AS VARCHAR), 'YYYYMMDD') AS sales_ymd, 
        TO_DATE(c.application_date, 'YYYYMMDD') AS application_date
    FROM receipt r
    JOIN customer c
    ON r.customer_id = c.customer_id
)
SELECT 
    customer_id,
    sales_ymd,
    application_date,
    sales_ymd - application_date AS progress_date
FROM date_table
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
7 rows affected.


customer_id,sales_ymd,application_date,progress_date
CS019515000097,2017-01-02 00:00:00+00:00,2014-11-24,"770 days, 0:00:00"
CS019515000097,2017-01-31 00:00:00+00:00,2014-11-24,"799 days, 0:00:00"
CS019515000097,2017-07-31 00:00:00+00:00,2014-11-24,"980 days, 0:00:00"
CS019515000097,2017-11-17 00:00:00+00:00,2014-11-24,"1089 days, 0:00:00"
CS019515000097,2017-11-30 00:00:00+00:00,2014-11-24,"1102 days, 0:00:00"
CS019515000097,2018-07-08 00:00:00+00:00,2014-11-24,"1322 days, 0:00:00"
CS019515000097,2019-06-30 00:00:00+00:00,2014-11-24,"1679 days, 0:00:00"


---
> S-071: レシート明細データ（receipt）の売上日（sales_ymd）に対し、顧客データ（customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [46]:
%%sql

WITH receipt_distinct AS (
    SELECT DISTINCT
        customer_id,
        sales_ymd
    FROM receipt
),
time_age_tbl AS(
    SELECT
        c.customer_id,
        r.sales_ymd,
        c.application_date,
        -- AGE関数で年月の差を取っている。
        AGE(TO_TIMESTAMP(CAST(r.sales_ymd AS VARCHAR), 'YYYYMMDD'), TO_TIMESTAMP(c.application_date, 'YYYYMMDD')) AS time_age
    FROM receipt_distinct r
    JOIN customer c
    ON r.customer_id = c.customer_id
)
SELECT
    customer_id, 
    sales_ymd, 
    application_date,
    EXTRACT(YEAR FROM time_age) * 12 + EXTRACT(MONTH FROM time_age) AS elapsed_months
FROM time_age_tbl
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sales_ymd,application_date,elapsed_months
CS017515000010,20171117,20150803,27
CS008413000003,20180524,20151228,28
CS019515000097,20190630,20141124,55
CS008515000005,20170714,20150216,28
CS026414000097,20170809,20150430,27
CS034514000008,20181012,20150807,38
CS029415000089,20180409,20150723,32
CS019411000012,20190314,20141213,51
CS015614000006,20190802,20150211,53
CS007515000053,20170712,20150325,27


---
> S-072: レシート明細データ（receipt）の売上日（sales_ymd）に対し、顧客データ（customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [2]:
%%sql

WITH receipt_distinct AS (
    SELECT DISTINCT
        customer_id,
        sales_ymd
    FROM receipt
),
time_age_tbl AS(
    SELECT
        c.customer_id,
        r.sales_ymd,
        c.application_date,
        -- AGE関数で年月の差を取っている。
        AGE(TO_TIMESTAMP(CAST(r.sales_ymd AS VARCHAR), 'YYYYMMDD'), TO_TIMESTAMP(c.application_date, 'YYYYMMDD')) AS time_age
    FROM receipt_distinct r
    JOIN customer c
    ON r.customer_id = c.customer_id
)
SELECT
    customer_id, 
    sales_ymd, 
    application_date,
    EXTRACT(YEAR FROM time_age) AS elapsed_years
FROM time_age_tbl
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sales_ymd,application_date,elapsed_years
CS017515000010,20171117,20150803,2
CS008413000003,20180524,20151228,2
CS019515000097,20190630,20141124,4
CS008515000005,20170714,20150216,2
CS026414000097,20170809,20150430,2
CS034514000008,20181012,20150807,3
CS029415000089,20180409,20150723,2
CS019411000012,20190314,20141213,4
CS015614000006,20190802,20150211,4
CS007515000053,20170712,20150325,2


---
> S-073: レシート明細データ（receipt）の売上日（sales_ymd）に対し、顧客データ（customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [6]:
%%sql

WITH date_table AS (
    SELECT
        r.customer_id AS customer_id,
        TO_TIMESTAMP(CAST(r.sales_ymd AS VARCHAR), 'YYYYMMDD') AS sales_ymd, 
        TO_TIMESTAMP(c.application_date, 'YYYYMMDD') AS application_date
    FROM receipt r
    JOIN customer c
    ON r.customer_id = c.customer_id
)
SELECT 
    customer_id,
    sales_ymd,
    application_date,
    EXTRACT(EPOCH FROM sales_ymd - application_date) AS progress_epoch
FROM date_table
WHERE customer_id = 'CS017515000010'
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sales_ymd,application_date,progress_epoch
CS017515000010,2018-01-02 00:00:00+00:00,2015-08-03 00:00:00+00:00,76291200.000000
CS017515000010,2018-01-02 00:00:00+00:00,2015-08-03 00:00:00+00:00,76291200.000000
CS017515000010,2019-02-28 00:00:00+00:00,2015-08-03 00:00:00+00:00,112752000.000000
CS017515000010,2017-07-31 00:00:00+00:00,2015-08-03 00:00:00+00:00,62899200.000000
CS017515000010,2019-02-28 00:00:00+00:00,2015-08-03 00:00:00+00:00,112752000.000000
CS017515000010,2017-07-31 00:00:00+00:00,2015-08-03 00:00:00+00:00,62899200.000000
CS017515000010,2017-11-18 00:00:00+00:00,2015-08-03 00:00:00+00:00,72403200.000000
CS017515000010,2017-11-17 00:00:00+00:00,2015-08-03 00:00:00+00:00,72316800.000000
CS017515000010,2017-11-17 00:00:00+00:00,2015-08-03 00:00:00+00:00,72316800.000000
CS017515000010,2017-11-18 00:00:00+00:00,2015-08-03 00:00:00+00:00,72403200.000000


---
> S-074: レシート明細データ（receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [19]:
%%sql

WITH monday_table AS (
    SELECT 
        sales_ymd,
        DATE_TRUNC('WEEK', TO_TIMESTAMP(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD')) AS monday
    FROM receipt
)
SELECT
    TO_DATE(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD'),
    EXTRACT(DAY FROM (TO_TIMESTAMP(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD') - monday)) AS progress_day,
    monday
FROM monday_table
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


to_date,progress_day,monday
2018-11-03,5,2018-10-29 00:00:00+00:00
2018-11-18,6,2018-11-12 00:00:00+00:00
2017-07-12,2,2017-07-10 00:00:00+00:00
2019-02-05,1,2019-02-04 00:00:00+00:00
2018-08-21,1,2018-08-20 00:00:00+00:00
2019-06-05,2,2019-06-03 00:00:00+00:00
2018-12-05,2,2018-12-03 00:00:00+00:00
2019-09-22,6,2019-09-16 00:00:00+00:00
2017-05-04,3,2017-05-01 00:00:00+00:00
2019-10-10,3,2019-10-07 00:00:00+00:00


---
> S-075: 顧客データ（customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [22]:
%%sql

WITH customer_random AS (
    SELECT
        customer_id,
        ROW_NUMBER() OVER(ORDER BY RANDOM()) AS row,
        COUNT(*) OVER() AS count
    FROM customer
)
SELECT
    customer_id
FROM customer_random
WHERE row <= count * 0.01
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id
CS038415000280
CS023514000113
CS015512000192
CS002413000739
CS003511000065
CS033715000078
CS001515000753
CS005315000153
CS027415000025
CS041315000007


---
> S-076: 顧客データ（customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [30]:
%%sql

WITH customer_count AS (
    SELECT
        COUNT(*) AS all_count,
        SUM(CASE
                WHEN gender_cd = '0' THEN 1
                ELSE 0
              END) AS male_count,
        SUM(CASE
                WHEN gender_cd = '1' THEN 1
                ELSE 0
              END) AS female_count,
        SUM(CASE
                WHEN gender_cd = '9' THEN 1
                ELSE 0
              END) AS unknow_count
    FROM customer
),
customer_random AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY gender_cd ORDER BY RANDOM()) AS row
    FROM customer
),
customer_gender_sample AS (
    SELECT
        cr.*
    FROM customer_random cr
    CROSS JOIN customer_count cc
    WHERE (cr.row <= cc.male_count * 0.1 AND cr.gender_cd = '0')
    OR (cr.row <= cc.female_count * 0.1 AND cr.gender_cd = '1')
    OR (cr.row <= cc.unknow_count * 0.1 AND cr.gender_cd = '9')
)
SELECT
    COUNT(*) AS sample_count
FROM customer_gender_sample
GROUP BY gender_cd
;
        

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


sample_count
298
1791
107


In [28]:
%%sql

    SELECT
        COUNT(*) AS all_count,
        SUM(CASE
                WHEN gender_cd = '0' THEN 1
                ELSE 0
              END) AS male_count,
        SUM(CASE
                WHEN gender_cd = '1' THEN 1
                ELSE 0
              END) AS female_count,
        SUM(CASE
                WHEN gender_cd = '9' THEN 1
                ELSE 0
              END) AS unknow_count
    FROM customer
    ;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


all_count,male_count,female_count,unknow_count
21971,2981,17918,1072


---
> S-077: レシート明細データ（receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [29]:
%%sql

WITH sum_receipt AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM receipt
    GROUP BY customer_id
),
avg_std_table AS (
    SELECT
        AVG(LOG(sum_amount)) AS avg,
        STDDEV_POP(LOG(sum_amount)) AS std
    FROM sum_receipt
)
SELECT
    s.customer_id,
    s.sum_amount AS sum_amount,
    LOG(s.sum_amount) AS log_sum_amount
FROM sum_receipt s
CROSS JOIN avg_std_table a
WHERE ABS(LOG(s.sum_amount) - a.avg) > 3 * a.std 
LIMIT 10;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


customer_id,sum_amount,log_sum_amount
ZZ000000000000,12395003,7.093246636217458


---
> S-078: レシート明細データ（receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [41]:
%%sql

WITH sum_receipt AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM receipt
    WHERE customer_id NOT LIKE 'Z%'
    GROUP BY customer_id
),
percent_25_75 AS (
    SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY sum_amount) AS sum_amount_25,
        PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY sum_amount) AS sum_amount_75
    FROM sum_receipt
)
SELECT
    s.customer_id,
    s.sum_amount AS sum_amount,
    p.sum_amount_25,
    p.sum_amount_75
FROM sum_receipt s
CROSS JOIN percent_25_75 p
WHERE s.sum_amount < p.sum_amount_25 - 1.5 * (p.sum_amount_75 - p.sum_amount_25)
OR s.sum_amount > p.sum_amount_75 + 1.5 * (p.sum_amount_75 - p.sum_amount_25)
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,sum_amount_25,sum_amount_75
CS013415000226,8362,548.25,3649.75
CS011415000041,9454,548.25,3649.75
CS014514000004,8872,548.25,3649.75
CS021514000008,12839,548.25,3649.75
CS014515000007,9763,548.25,3649.75
CS040415000220,10158,548.25,3649.75
CS028415000161,8465,548.25,3649.75
CS034515000173,10074,548.25,3649.75
CS022515000065,12903,548.25,3649.75
CS007514000094,15735,548.25,3649.75


---
> S-079: 商品データ（product）の各項目に対し、欠損数を確認せよ。

In [42]:
%%sql

SELECT
    SUM(CASE
            WHEN product_cd IS NULL THEN 1
            ELSE 0
        END) AS product_cd,
    SUM(CASE
            WHEN category_major_cd IS NULL THEN 1
            ELSE 0
        END) AS category_major_cd,
    SUM(CASE
            WHEN category_medium_cd IS NULL THEN 1
            ELSE 0
        END) AS category_medium_cd,
    SUM(CASE
            WHEN category_small_cd IS NULL THEN 1
            ELSE 0
        END) AS category_small_cd,
    SUM(CASE
            WHEN unit_price IS NULL THEN 1
            ELSE 0
        END) AS unit_price,
    SUM(CASE
            WHEN unit_cost IS NULL THEN 1
            ELSE 0
        END) AS unit_cost
FROM product
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,0,0,0,7,7


---
> S-080: 商品データ（product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [3]:
%%sql

DROP TABLE IF EXISTS product_not_null;
CREATE TABLE product_not_null AS (
    SELECT 
        * 
    FROM product
    WHERE unit_price IS NOT NULL 
    AND unit_cost IS NOT NULL
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10023 rows affected.


[]

In [46]:
%%sql

SELECT
    COUNT(1) AS count
FROM product
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
10030


In [47]:
%%sql

SELECT
    COUNT(1) AS count
FROM product_not_null
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
10023


---
> S-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [12]:
%%sql


DROP TABLE IF EXISTS product_mean_imp;

CREATE TABLE product_mean_imp AS (
    WITH price_cost_mean AS (
        SELECT
            ROUND(AVG(unit_price)) as avg_price,
            ROUND(AVG(unit_cost)) as avg_cost
        FROM product
    )
    SELECT 
        product_cd, 
        category_major_cd, 
        category_medium_cd, 
        category_small_cd, 
        COALESCE(unit_price, avg_price) AS unit_price,
        COALESCE(unit_cost, avg_cost) AS unit_cost
    FROM product
    CROSS JOIN price_cost_mean
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10030 rows affected.


[]

In [11]:
%%sql

SELECT
    SUM(CASE
            WHEN product_cd IS NULL THEN 1
            ELSE 0
        END) AS product_cd,
    SUM(CASE
            WHEN category_major_cd IS NULL THEN 1
            ELSE 0
        END) AS category_major_cd,
    SUM(CASE
            WHEN category_medium_cd IS NULL THEN 1
            ELSE 0
        END) AS category_medium_cd,
    SUM(CASE
            WHEN category_small_cd IS NULL THEN 1
            ELSE 0
        END) AS category_small_cd,
    SUM(CASE
            WHEN unit_price IS NULL THEN 1
            ELSE 0
        END) AS unit_price,
    SUM(CASE
            WHEN unit_cost IS NULL THEN 1
            ELSE 0
        END) AS unit_cost
FROM product_mean_imp
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,0,0,0,0,0


---
> S-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [14]:
%%sql

DROP TABLE IF EXISTS product_center_imp;

CREATE TABLE product_center_imp AS (
    WITH price_cost_center AS (
        SELECT
            ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY unit_price)) as center_price,
            ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY unit_cost)) as center_cost
        FROM product
    )
    SELECT 
        product_cd, 
        category_major_cd, 
        category_medium_cd, 
        category_small_cd, 
        COALESCE(unit_price, center_price) AS unit_price,
        COALESCE(unit_cost, center_cost) AS unit_cost
    FROM product
    CROSS JOIN price_cost_center
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10030 rows affected.


[]

In [17]:
%%sql

SELECT
    SUM(CASE
            WHEN product_cd IS NULL THEN 1
            ELSE 0
        END) AS product_cd,
    SUM(CASE
            WHEN category_major_cd IS NULL THEN 1
            ELSE 0
        END) AS category_major_cd,
    SUM(CASE
            WHEN category_medium_cd IS NULL THEN 1
            ELSE 0
        END) AS category_medium_cd,
    SUM(CASE
            WHEN category_small_cd IS NULL THEN 1
            ELSE 0
        END) AS category_small_cd,
    SUM(CASE
            WHEN unit_price IS NULL THEN 1
            ELSE 0
        END) AS unit_price,
    SUM(CASE
            WHEN unit_cost IS NULL THEN 1
            ELSE 0
        END) AS unit_cost
FROM product_center_imp
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,0,0,0,0,0


---
> S-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [23]:
%%sql

DROP TABLE IF EXISTS product_small_center_imp;

CREATE TABLE product_small_center_imp AS (
    WITH price_cost_center AS (
        SELECT
            category_small_cd,
            ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY unit_price)) as center_price,
            ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY unit_cost)) as center_cost
        FROM product
        GROUP BY category_small_cd
    )
    SELECT 
        p.product_cd, 
        p.category_major_cd, 
        p.category_medium_cd, 
        p.category_small_cd, 
        COALESCE(p.unit_price, pcc.center_price) AS unit_price,
        COALESCE(p.unit_cost, pcc.center_cost) AS unit_cost
    FROM product p
    JOIN price_cost_center pcc
    ON p.category_small_cd = pcc.category_small_cd
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10030 rows affected.


[]

In [24]:
%%sql

SELECT
    SUM(CASE
            WHEN product_cd IS NULL THEN 1
            ELSE 0
        END) AS product_cd,
    SUM(CASE
            WHEN category_major_cd IS NULL THEN 1
            ELSE 0
        END) AS category_major_cd,
    SUM(CASE
            WHEN category_medium_cd IS NULL THEN 1
            ELSE 0
        END) AS category_medium_cd,
    SUM(CASE
            WHEN category_small_cd IS NULL THEN 1
            ELSE 0
        END) AS category_small_cd,
    SUM(CASE
            WHEN unit_price IS NULL THEN 1
            ELSE 0
        END) AS unit_price,
    SUM(CASE
            WHEN unit_cost IS NULL THEN 1
            ELSE 0
        END) AS unit_cost
FROM product_small_center_imp
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,0,0,0,0,0


---
> S-084: 顧客データ（customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [35]:
%%sql

DROP TABLE IF EXISTS customer_2019_sales;

CREATE TABLE customer_2019_sales AS (
    WITH sum_amount_all_2019 AS (
        SELECT
            customer_id,
            SUM(amount) AS sum_amount_all,
            SUM(CASE
                    WHEN (sales_ymd >= 20190101 AND sales_ymd <= 20191231) THEN amount
                    ELSE 0
                END) AS sum_amount_2019
        FROM receipt
        GROUP BY customer_id
    )
    SELECT 
        c.customer_id,
        COALESCE(s.sum_amount_all, 0) AS sum_amount_all,
        COALESCE(s.sum_amount_2019, 0) AS sum_amount_2019,
        COALESCE(1.0 * sum_amount_2019 / sum_amount_all, 0) AS sales_rate
    FROM customer c
    LEFT JOIN sum_amount_all_2019 s
    ON c.customer_id = s.customer_id
);

SELECT
    *
FROM customer_2019_sales
WHERE sales_rate > 0
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21971 rows affected.
10 rows affected.


customer_id,sum_amount_all,sum_amount_2019,sales_rate
CS031415000172,5088,2971,0.58392295597484276730
CS015414000103,3122,874,0.27994875080076873799
CS011215000048,3444,248,0.07200929152148664344
CS029415000023,5167,3767,0.72904973872653377201
CS035415000029,7504,5823,0.77598614072494669510
CS023513000066,771,208,0.26977950713359273671
CS035513000134,1565,463,0.29584664536741214058
CS001515000263,216,216,1.00000000000000000000
CS006415000279,229,229,1.00000000000000000000
CS031415000106,7741,215,0.02777418938121689704


---
> S-085: 顧客データ（customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [38]:
%%sql

DROP TABLE IF EXISTS customer_link_geocode;

CREATE TABLE customer_link_geocode AS (
    WITH geocode_mean AS (
        SELECT
            postal_cd,
            AVG(longitude) AS avg_longitude,
            AVG(latitude) AS avg_latitude
        FROM geocode
        GROUP BY postal_cd
    )
    SELECT
        *
    FROM customer
    JOIN geocode_mean g
    USING(postal_cd)
);

SELECT
    *
FROM customer_link_geocode
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21971 rows affected.
10 rows affected.


postal_cd,customer_id,customer_name,gender_cd,gender,birth_day,age,address,application_store_cd,application_date,status_cd,avg_longitude,avg_latitude
259-1113,CS021313000114,大野 あや子,1,女性,1981-04-29,37,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.3177900000000000,35.4135800000000000
136-0076,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.8350200000000000,35.6719300000000000
151-0053,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.6896500000000000,35.6737400000000000
245-0016,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,139.4836000000000000,35.3912500000000000
144-0055,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,139.7077500000000000,35.5408400000000000
136-0073,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,東京都江東区北砂**********,S13015,20150722,B-20100609-B,139.8360100000000000,35.6781800000000000
136-0073,CS015804000004,松谷 米蔵,0,男性,1931-05-02,87,東京都江東区北砂**********,S13015,20150607,0-00000000-0,139.8360100000000000,35.6781800000000000
276-0022,CS007403000016,依田 満,0,男性,1975-08-18,43,千葉県八千代市上高野**********,S12007,20150914,0-00000000-0,140.1326000000000000,35.7326400000000000
154-0015,CS035614000014,板倉 菜々美,1,女性,1954-07-16,64,東京都世田谷区桜新町**********,S13035,20150804,0-00000000-0,139.6429700000000000,35.6288900000000000
223-0062,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9,139.6294600000000000,35.5537000000000000


---
> S-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [45]:
%%sql

SELECT
    c.customer_id,
    c.address,
    s.address,
    -- ラジアンになおす
    6371 * ACOS(SIN(RADIANS(s.latitude)) * SIN(RADIANS(c.avg_latitude)) + 
        COS(RADIANS(s.latitude)) * COS(RADIANS(c.avg_latitude)) * COS(RADIANS(s.longitude) - RADIANS(c.avg_longitude))) AS distance
FROM customer_link_geocode c
JOIN store s
ON c.application_store_cd = s.store_cd
LIMIT 10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,address,address_1,distance
CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.3944087027154715
CS037613000071,東京都江東区南砂**********,東京都江東区南砂一丁目,1.4511822099658445
CS031415000172,東京都渋谷区代々木**********,東京都渋谷区初台二丁目,0.4117334789298223
CS028811000001,神奈川県横浜市泉区和泉町**********,神奈川県横浜市瀬谷区二ツ橋町,8.065196026704987
CS001215000145,東京都大田区仲六郷**********,東京都大田区仲六郷二丁目,1.2684209720729687
CS015414000103,東京都江東区北砂**********,東京都江東区南砂二丁目,1.449673414532165
CS015804000004,東京都江東区北砂**********,東京都江東区南砂二丁目,1.449673414532165
CS007403000016,千葉県八千代市上高野**********,千葉県佐倉市上志津,1.9208032538419055
CS035614000014,東京都世田谷区桜新町**********,東京都世田谷区用賀四丁目,1.0214681484997588
CS011215000048,神奈川県横浜市港北区日吉本町**********,神奈川県横浜市港北区日吉本町四丁目,0.8182767808775093


---
> S-087: 顧客データ（customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [58]:
%%sql

DROP TABLE IF EXISTS customer_nayori;

CREATE TABLE customer_nayori AS (
    WITH customer_name_postal AS (
        SELECT
            *
        FROM customer
        GROUP BY customer_id, customer_name, postal_cd
    ),
    customer_amount AS (
        SELECT
            customer_id,
            SUM(amount) as sum_amount
        FROM receipt
        GROUP BY customer_id
    ),
    sum_amount_ranking AS (
        SELECT
            customer_id,
            customer_name,
            postal_cd,
            COALESCE(sum_amount, 0) AS sum_amount,
            -- customer_nameとpostal_cdの組み合わせないでsum_amountを降順に並べ、同じものがあったらcustomer_idを使って昇順にしてランキングする
            ROW_NUMBER() OVER(
                PARTITION BY customer_name, postal_cd 
                ORDER BY sum_amount desc, customer_id ) AS ranking
        FROM customer_name_postal
        LEFT JOIN customer_amount
        USING(customer_id)
    )
    SELECT c.*
    FROM
        customer c
    JOIN
        sum_amount_ranking r
    ON
        c.customer_id = r.customer_id
        AND r.ranking = 1
        
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21941 rows affected.


[]

In [61]:
%%sql

WITH customer_count AS (
    SELECT
        COUNT(1) AS customer_count
    FROM customer
),
customer_nayori_count AS (
    SELECT
        COUNT(1) AS customer_nayori_count
    FROM customer_nayori
)
SELECT
    c.customer_count,
    n.customer_nayori_count,
    c.customer_count - n.customer_nayori_count AS duplicate_count
FROM customer_count c
CROSS JOIN customer_nayori_count n
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


customer_count,customer_nayori_count,duplicate_count
21971,21941,30


---
> S-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [64]:
%%sql

DROP TABLE IF EXISTS customer_nayori_integrated;

CREATE TABLE customer_nayori_integrated AS (
    SELECT
        c.customer_id,
        n.customer_id AS integrated_customer_id
    FROM customer c
    LEFT JOIN customer_nayori n
    ON c.customer_name = n.customer_name
    AND c.postal_cd = n.postal_cd
);

SELECT
    COUNT(1) AS unique_diff_count
FROM customer_nayori_integrated
WHERE customer_id != integrated_customer_id
;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21971 rows affected.
1 rows affected.


unique_diff_count
30


---
> S-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [75]:
%%sql

DROP TABLE IF EXISTS customer_train_test;

CREATE TABLE customer_train_test AS (
    WITH customer_amount AS (
        SELECT
            customer_id,
            SUM(amount) as sum_amount
        FROM receipt
        GROUP BY customer_id
    ),
    customer_id_not_0 AS (
        SELECT
            customer_id
        FROM customer_amount
        WHERE sum_amount > 0
    ),
    customer_id_train_test AS (
        SELECT
            customer_id,
            ROW_NUMBER() OVER(ORDER BY RANDOM()) AS row,
            COUNT(*) OVER() AS count
        FROM customer_id_not_0
    )
    SELECT
        customer_id,
        CASE
            WHEN row <= count * 0.8 THEN 'train'
            ELSE 'test'
        END AS train_test
    FROM customer_id_train_test
);

DROP TABLE IF EXISTS customer_train;
CREATE TABLE customer_train AS (
    SELECT
        customer.*
    FROM
        customer_train_test
    JOIN
        customer 
    USING(customer_id)
    WHERE
        train_test = 'train'
);

DROP TABLE IF EXISTS customer_test;
CREATE TABLE customer_test AS (
    SELECT
        customer.*
    FROM
        customer_train_test
    JOIN
        customer 
    USING(customer_id)
    WHERE
        train_test = 'test'
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
8307 rows affected.
Done.
6644 rows affected.
Done.
1662 rows affected.


[]

---
> S-090: レシート明細データ（receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [98]:
%%sql

DROP TABLE IF EXISTS receipt_monthly_base;

CREATE TABLE receipt_monthly_base AS (
    SELECT
        EXTRACT(YEAR FROM (TO_TIMESTAMP(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD'))) AS year,
        EXTRACT(MONTH FROM (TO_TIMESTAMP(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD'))) AS month,
        SUM(amount) AS sum_amount_monthly
    FROM receipt
    GROUP BY year, month  
);

DROP TABLE IF EXISTS receipt_monthly_train_test_1;

CREATE TABLE receipt_monthly_train_test_1 AS (
    WITH receipt_monthly_train_test AS (
        SELECT
            year,
            month,
            sum_amount_monthly,
            ROW_NUMBER() OVER(ORDER BY RANDOM()) AS row,
            COUNT(*) OVER() AS count
        FROM receipt_monthly_base
    ),
    receipt_monthly_train_test_2 AS (
        SELECT
            year,
            month,
            sum_amount_monthly,
            CASE
                WHEN row <= 12 THEN 'train'
                WHEN 12 < row AND row <= 18 THEN 'test'
            END AS train_test
        FROM receipt_monthly_train_test
    )
    SELECT
        year,
        month,
        sum_amount_monthly,
        train_test
    FROM receipt_monthly_train_test_2
    WHERE train_test IS NOT NULL
);

DROP TABLE IF EXISTS receipt_monthly_train_test_2;

CREATE TABLE receipt_monthly_train_test_2 AS (
    WITH receipt_monthly_train_test AS (
        SELECT
            year,
            month,
            sum_amount_monthly,
            ROW_NUMBER() OVER(ORDER BY RANDOM()) AS row,
            COUNT(*) OVER() AS count
        FROM receipt_monthly_base
    ),
    receipt_monthly_train_test_2 AS (
        SELECT
            year,
            month,
            sum_amount_monthly,
            CASE
                WHEN row <= 12 THEN 'train'
                WHEN 12 < row AND row <= 18 THEN 'test'
            END AS train_test
        FROM receipt_monthly_train_test
    )
    SELECT
        year,
        month,
        sum_amount_monthly,
        train_test
    FROM receipt_monthly_train_test_2
    WHERE train_test IS NOT NULL
);

DROP TABLE IF EXISTS receipt_monthly_train_test_3;

CREATE TABLE receipt_monthly_train_test_3 AS (
    WITH receipt_monthly_train_test AS (
        SELECT
            year,
            month,
            sum_amount_monthly,
            ROW_NUMBER() OVER(ORDER BY RANDOM()) AS row,
            COUNT(*) OVER() AS count
        FROM receipt_monthly_base
    ),
    receipt_monthly_train_test_2 AS (
        SELECT
            year,
            month,
            sum_amount_monthly,
            CASE
                WHEN row <= 12 THEN 'train'
                WHEN 12 < row AND row <= 18 THEN 'test'
            END AS train_test
        FROM receipt_monthly_train_test
    )
    SELECT
        year,
        month,
        sum_amount_monthly,
        train_test
    FROM receipt_monthly_train_test_2
    WHERE train_test IS NOT NULL
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
34 rows affected.
Done.
18 rows affected.
Done.
18 rows affected.
Done.
18 rows affected.


[]

In [99]:
%%sql

SELECT
    *
FROM receipt_monthly_train_test_1
;

*  postgresql://padawan:***@db:5432/dsdojo_db
18 rows affected.


year,month,sum_amount_monthly,train_test
2018,1,944509,train
2019,1,1064085,train
2018,4,937099,train
2018,3,946588,train
2017,3,962945,train
2018,6,1012329,train
2019,5,1111985,train
2018,5,1004438,train
2018,9,977114,train
2018,7,1058472,train


In [100]:
%%sql

SELECT
    *
FROM receipt_monthly_train_test_2
;

*  postgresql://padawan:***@db:5432/dsdojo_db
18 rows affected.


year,month,sum_amount_monthly,train_test
2019,7,1118371,train
2018,7,1058472,train
2017,8,954836,train
2018,3,946588,train
2019,5,1111985,train
2019,10,1143062,train
2018,10,1069939,train
2018,2,864128,train
2017,6,894242,train
2019,6,1089063,train


In [101]:
%%sql

SELECT
    *
FROM receipt_monthly_train_test_3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
18 rows affected.


year,month,sum_amount_monthly,train_test
2018,6,1012329,train
2019,1,1064085,train
2017,2,764413,train
2017,12,939654,train
2018,9,977114,train
2018,10,1069939,train
2018,1,944509,train
2019,7,1118371,train
2019,5,1111985,train
2017,10,905739,train


---
> S-091: 顧客データ（customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [110]:
%%sql

DROP TABLE IF EXISTS customer_under_sampling;

CREATE TABLE customer_under_sampling AS (
    WITH customer_amount AS (
        SELECT
            customer_id,
            SUM(amount) as sum_amount
        FROM receipt
        GROUP BY customer_id
    ),
    customer_add_sum_amount AS (
        SELECT
            c.*,
            COALESCE(sum_amount,0) AS sum_amount
        FROM customer c
        LEFT JOIN customer_amount
        USING(customer_id)
    ),
    customer_add_amount_flag AS (
        SELECT
            *,
            CASE
                WHEN sum_amount > 0 THEN 1
                ELSE 0
            END AS amount_flag
        FROM customer_add_sum_amount
    ),
    customer_add_amount_flag_row AS (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY amount_flag ORDER BY RANDOM()) AS row
        FROM customer_add_amount_flag
    )
    SELECT
        *
    FROM customer_add_amount_flag_row
    WHERE row <= (SELECT COUNT(1) FROM customer_add_amount_flag WHERE amount_flag = 0)
    AND row <= (SELECT COUNT(1) FROM customer_add_amount_flag WHERE amount_flag = 1)
);
    

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
16612 rows affected.


[]

In [112]:
%%sql

SELECT
    COUNT(1) AS count
FROM customer_under_sampling
GROUP BY amount_flag
;

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


count
8306
8306


---
> S-092: 顧客データ（customer）の性別について、第三正規形へと正規化せよ。

In [3]:
%%sql

DROP TABLE IF EXISTS customer_3;

CREATE TABLE customer_3 AS (
    SELECT
        customer_id,
        customer_name,
        gender_cd,
        birth_day,
        age,
        postal_cd,
        address,
        application_store_cd,
        application_date,
        status_cd
    FROM customer
);

DROP TABLE IF EXISTS gender_table;

CREATE TABLE gender_table AS (
    SELECT DISTINCT
        gender_cd,
        gender
    FROM customer
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21971 rows affected.
Done.
3 rows affected.


[]

In [5]:
%%sql

SELECT
    *
FROM gender_table
;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


gender_cd,gender
0,男性
9,不明
1,女性


---
> S-093: 商品データ（product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [21]:
%%sql

DROP TABLE IF EXISTS product_category;

CREATE TABLE product_category AS (
    SELECT
        p.product_cd,
        p.category_major_cd,
        p.category_medium_cd,
        p.category_small_cd,
        p.unit_price,
        p.unit_cost,
        c.category_major_name,
        c.category_medium_name,
        c.category_small_name
    FROM product p
    JOIN category c
    ON p.category_major_cd = c.category_major_cd
    AND p.category_medium_cd = c.category_medium_cd
    AND p.category_small_cd = c.category_small_cd
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10030 rows affected.


[]

---
> S-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること（COPYコマンドの権限は付与済み）。
> 
> |出力先|
> |:--:|
> |/tmp/data|
> 
> ※"/tmp/data"を指定することでJupyterの"/work/data"と共有されるようになっている。

In [22]:
%%sql

COPY product_category TO '/tmp/data/product_category.csv' WITH CSV HEADER ENCODING 'UTF-8';

*  postgresql://padawan:***@db:5432/dsdojo_db
10030 rows affected.


[]

---
> S-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> PostgreSQLではShift_JISを指定することでCP932相当となる。ファイル出力先のパスは以下のようにすること（COPYコマンドの権限は付与済み）。
> 
> |出力先|
> |:--:|
> |/tmp/data|
> 
> ※"/tmp/data"を指定することでJupyterの"/work/data"と共有されるようになっている。

In [8]:
%%sql

COPY product_category TO '/tmp/data/product_category_SJIS.csv' WITH CSV HEADER ENCODING 'SJIS';

*  postgresql://padawan:***@db:5432/dsdojo_db
10030 rows affected.


[]

---
> S-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること（COPYコマンドの権限は付与済み）。
> 
> |出力先|
> |:--:|
> |/tmp/data|
> 
> ※"/tmp/data"を指定することでJupyterの"/work/data"と共有されるようになっている。

In [23]:
%%sql

COPY product_category TO '/tmp/data/product_category_no_header.csv' WITH CSV ENCODING 'UTF-8';

*  postgresql://padawan:***@db:5432/dsdojo_db
10030 rows affected.


[]

---
> S-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [25]:
%%sql

DROP TABLE IF EXISTS product_category_97;

CREATE TABLE product_category_97 (
        product_cd           VARCHAR(10),
        category_major_cd    VARCHAR(2),
        category_medium_cd   VARCHAR(4),
        category_small_cd    VARCHAR(6),
        unit_price           INTEGER,
        unit_cost            INTEGER,
        category_major_name  VARCHAR(20),
        category_medium_name VARCHAR(20),
        category_small_name  VARCHAR(20)
);

COPY product_category_97 FROM '/tmp/data/product_category.csv' WITH CSV HEADER ENCODING 'UTF-8';
SELECT
    *
FROM product_category_97
LIMIT 3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
10030 rows affected.
3 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


---
> S-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [24]:
%%sql

DROP TABLE IF EXISTS product_category_98;
CREATE TABLE product_category_98 (
        product_cd           VARCHAR(10),
        category_major_cd    VARCHAR(2),
        category_medium_cd   VARCHAR(4),
        category_small_cd    VARCHAR(6),
        unit_price           INTEGER,
        unit_cost            INTEGER,
        category_major_name  VARCHAR(20),
        category_medium_name VARCHAR(20),
        category_small_name  VARCHAR(20)
);

COPY product_category_98 FROM '/tmp/data/product_category_no_header.csv' WITH CSV ENCODING 'UTF-8';
SELECT
    *
FROM product_category_98
LIMIT 3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
10030 rows affected.
3 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


---
> S-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること（COPYコマンドの権限は付与済み）。
> 
> |出力先|
> |:--:|
> |/tmp/data|
> 
> ※"/tmp/data"を指定することでJupyterの"/work/data"と共有されるようになっている。


In [14]:
%%sql

COPY product_category TO '/tmp/data/product_category.tsv' WITH CSV HEADER DELIMITER E'\t' ENCODING 'UTF-8';

*  postgresql://padawan:***@db:5432/dsdojo_db
10030 rows affected.


[]

---
> S-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [26]:
%%sql

DROP TABLE IF EXISTS product_category_100;

CREATE TABLE product_category_100 (
        product_cd           VARCHAR(10),
        category_major_cd    VARCHAR(2),
        category_medium_cd   VARCHAR(4),
        category_small_cd    VARCHAR(6),
        unit_price           INTEGER,
        unit_cost            INTEGER,
        category_major_name  VARCHAR(20),
        category_medium_name VARCHAR(20),
        category_small_name  VARCHAR(20)
);
COPY product_category_100 FROM '/tmp/data/product_category.tsv' WITH CSV HEADER DELIMITER E'\t' ENCODING 'UTF-8';
SELECT
    *
FROM product_category_100
LIMIT 3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
10030 rows affected.
3 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
P040101001,04,0401,040101,198,149,惣菜,御飯類,弁当類
P040101002,04,0401,040101,218,164,惣菜,御飯類,弁当類
P040101003,04,0401,040101,230,173,惣菜,御飯類,弁当類


# これで１００本終わりです。おつかれさまでした！